In [1]:
pip install torch

     |████████████████████████████████| 734.6MB 48kB/s s eta 0:00:01    |█▉                              | 42.3MB 22.9MB/s eta 0:00:31     |███████████▏                    | 255.3MB 29.0MB/s eta 0:00:17     |██████████████▍                 | 331.3MB 43.9MB/s eta 0:00:10     |███████████████▌                | 355.3MB 43.9MB/s eta 0:00:09     |████████████████▍               | 376.8MB 62.4MB/s eta 0:00:06     |████████████████▌               | 379.6MB 62.4MB/s eta 0:00:06     |██████████████████▏             | 416.1MB 30.4MB/s eta 0:00:11     |█████████████████████▏          | 486.3MB 47.1MB/s eta 0:00:06     |█████████████████████▉          | 501.8MB 32.7MB/s eta 0:00:08     |███████████████████████▎        | 534.1MB 54kB/s eta 1:01:21:59:14     |███████████████████████▊        | 543.7MB 54kB/s eta 0:58:25     |█████████████████████████▌      | 584.9MB 64.9MB/s eta 0:00:03
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install itertools

  ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pickle

  ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
import numpy as np
import torch

import itertools
import pickle

# This will be our input ---> x
text = 'MathMathMathMathMath'

character_list = list(set(text))   # get all of the unique letters in our text variable
vocabulary_size = len(character_list)   # count the number of unique elements
character_dictionary = {'h': 0, 'a': 1, 't': 2, 'M': 3}
# {char:e for e, char in enumerate(character_list)}  # create a dictionary mapping each unique char to a number
encoded_chars = [character_dictionary[char] for char in text] #integer representation of our vocabulary

print(character_dictionary)
print(encoded_chars)
def one_hot_encode(encoded, vocab_size):
    result = torch.zeros((len(encoded), vocab_size))
    for i, idx in enumerate(encoded):
        result[i, idx] = 1.0
    return result

# One hot encode our encoded charactes
batch_size = 2
seq_length = 3
num_samples = (len(encoded_chars) - 1) // seq_length # time lag of 1 for creating the labels
vocab_size = 4

data = one_hot_encode(encoded_chars[:seq_length*num_samples], vocab_size).reshape((num_samples, seq_length, vocab_size))
num_batches = len(data) // batch_size
X = data[:num_batches*batch_size].reshape((num_batches, batch_size, seq_length, vocab_size))
# swap batch_size and seq_length axis to make later access easier
X = X.transpose(1, 2)
X.shape
# However, we typically need the batch to be first so we would need to reshape our dataset --. S x B x V
X, X.shape

# +1 shift the labels by one so that given the previous letter the char we should predict would be or next char
labels = one_hot_encode(encoded_chars[1:seq_length*num_samples+1], vocab_size)
y = labels.reshape((num_batches, batch_size, seq_length, vocab_size))
y = y.transpose(1, 2) # transpose the first and second index
y,y.shape

torch.manual_seed(1) # reproducibility

####  Define the network parameters:
hiddenSize = 2 # network size, this can be any number (depending on your task)
numClass = 4 # this is the same as our vocab_size

#### Weight matrices for our inputs
Wz = Variable(torch.randn(vocab_size, hiddenSize), requires_grad=True)
Wr = Variable(torch.randn(vocab_size, hiddenSize), requires_grad=True)
Wh = Variable(torch.randn(vocab_size, hiddenSize), requires_grad=True)

## Intialize the hidden state
# this is for demonstration purposes only, in the actual model it will be initiated during training a loop over the
# the number of bacthes and updated before passing to the next GRU cell.
h_t_demo = torch.zeros(batch_size, hiddenSize)

#### Weight matrices for our hidden layer
Uz = Variable(torch.randn(hiddenSize, hiddenSize), requires_grad=True)
Ur = Variable(torch.randn(hiddenSize, hiddenSize), requires_grad=True)
Uh = Variable(torch.randn(hiddenSize, hiddenSize), requires_grad=True)

#### bias vectors for our hidden layer
bz = Variable(torch.zeros(hiddenSize), requires_grad=True)
br = Variable(torch.zeros(hiddenSize), requires_grad=True)
bh = Variable(torch.zeros(hiddenSize), requires_grad=True)

#### Output weights
Wy = Variable(torch.randn(hiddenSize, numClass), requires_grad=True)
by = Variable(torch.zeros(numClass), requires_grad=True)

print(f'Weight Matrix for Wz:\n {Wz,Wz.size()}','\n')
print(f'Weight Matrix for Uz:\n {Uz, Uz.size()}','\n')
print(f'Weight Matrix for h_t_demo:\n {h_t_demo,h_t_demo.size()}','\n')
print(f'Bias vector for bz: \n {bz, bz.size()}','\n')
torch.matmul(X[0][0], Wz) # This can be easily handled in pytorch using the mm or matmul(handles broadcasting)
torch.matmul(Uz, h_t_demo), Uz
bz
z_inner = torch.matmul(X,Wz) +  torch.matmul(Uz,h_t_demo) + bz
z_inner[0][0]
z = torch.sigmoid(z_inner)
z[0][0]

r = torch.sigmoid(torch.matmul(X,Wr) +  torch.matmul(Ur,h_t_demo) + br)
r[0][0]  # first bactch sequence
torch.matmul(X[0][0], Wh)
torch.matmul(r[0][0] * h_t_demo, Uh)
bh
h_inner_tilde = torch.matmul(X,Wh) +  r*torch.matmul(Uh,h_t_demo) + bh
h_inner_tilde[0][0]

h_tilde = torch.tanh(h_inner_tilde)
h_tilde[0]

ht_1 = z *h_t_demo + (1-z)* h_tilde
ht_1[0][0]
# h gets updated and then we calculate for the next
h_t_1 = []
h = h_t_demo
for i,sequence in enumerate(X[0]):   # iterate over each sequence in the batch to calculate the hidden state h
    z = torch.sigmoid(torch.matmul(sequence, Wz) + torch.matmul(h, Uz) + bz)
    r = torch.sigmoid(torch.matmul(sequence, Wr) + torch.matmul(h, Ur) + br)
    h_tilde = torch.tanh(torch.matmul(sequence, Wh) + torch.matmul(r * h, Uh) + bh)
    h = z * h + (1 - z) * h_tilde
    h_t_1.append(h)
    print(f'h{i}:{h}')
h_t_1 = torch.stack(h_t_1)
h_t_minus_1 = torch.tensor([[ 0.7565, -0.3472],
        [-0.1355, -0.2040]])
X[0][0], X[0][1] # second sequence in batch 1
h = h_t_minus_1
z = torch.sigmoid(torch.matmul(X[0][1], Wz) + torch.matmul(h, Uz) + bz)
r = torch.sigmoid(torch.matmul(X[0][1], Wr) + torch.matmul(h, Ur) + br)
h_tilde = torch.tanh(torch.matmul(X[0][1], Wh) + torch.matmul(r * h, Uh) + bh)
hh = z * h + (1 - z) * h_tilde
hh
ht_2 = [] # stores the calculated h for each input x
h = torch.zeros(batch_size, hiddenSize) # intitalizes the hidden state
for batch in range(num_batches):  # this loops over the batches
    x = X[batch]
    for sequence in x: # iterates over the sequences in each batch
        z = torch.sigmoid(torch.matmul(sequence, Wz) + torch.matmul(h, Uz) + bz)
        r = torch.sigmoid(torch.matmul(sequence, Wr) + torch.matmul(h, Ur) + br)
        h_tilde = torch.tanh(torch.matmul(sequence, Wh) + torch.matmul(r * h, Uh) + bh)
        h = z * h + (1 - z) * h_tilde
        ht_2.append(h)
ht_2 = torch.stack(ht_2)
ht_2
# This is the same
fully_connected = torch.matmul(ht_2, Wy) + by
fully_connected[0]
fully_connected
ylin_max = (fully_connected - fully_connected.max()) # first sequence in batch 1
ylin_max[0]
exp = ylin_max.exp()
exp[0]
exp_sum = exp[0].sum(dim=1,keepdim=True).reshape(-1,1) # the max value in each element within the sequence
exp_sum
exp[0]/exp_sum
ht_2 = []  # stores the calculated h for each input x
outputs = []
h = torch.zeros(batch_size, hiddenSize)  # intitalizes the hidden state
for i in range(num_batches):  # this loops over the batches
    x = X[i]
    for i, sequence in enumerate(x):  # iterates over the sequences in each batch
        z = torch.sigmoid(torch.matmul(sequence, Wz) + torch.matmul(h, Uz) + bz)
        r = torch.sigmoid(torch.matmul(sequence, Wr) + torch.matmul(h, Ur) + br)
        h_tilde = torch.tanh(torch.matmul(sequence, Wh) + torch.matmul(r * h, Uh) + bh)
        h = z * h + (1 - z) * h_tilde

        # Linear layer
        y_linear = torch.matmul(h, Wy) + by

        # Softmax activation function
        y_t = F.softmax(y_linear, dim=1)

        ht_2.append(h)
        outputs.append(y_t)

ht_2 = torch.stack(ht_2)
outputs = torch.stack(outputs)
outputs

hh = torch.tensor([[ 0.7948, -0.8400],
         [-0.3061, -0.7358]])
fcc = torch.matmul(hh, Wy) + by
F.softmax(fcc, dim=1)

def gru(x, h):
    outputs = []
    for i,sequence in enumerate(x): # iterates over the sequences in each batch
        z = torch.sigmoid(torch.matmul(sequence, Wz) + torch.matmul(h, Uz) + bz)
        r = torch.sigmoid(torch.matmul(sequence, Wr) + torch.matmul(h, Ur) + br)
        h_tilde = torch.tanh(torch.matmul(sequence, Wh) + torch.matmul(r * h, Uh) + bh)
        h = z * h + (1 - z) * h_tilde

        # Linear layer
        y_linear = torch.matmul(h, Wy) + by

        # Softmax activation function
        y_t = F.softmax(y_linear, dim=1)

        outputs.append(y_t)
    return torch.stack(outputs), h


def sample(primer, length_chars_predict):
    word = primer

    primer_dictionary = [character_dictionary[char] for char in word]
    test_input = one_hot_encode(primer_dictionary, vocab_size)

    h = torch.zeros(1, hiddenSize)

    for i in range(length_chars_predict):
        outputs, h = gru(test_input, h)
        choice = np.random.choice(vocab_size, p=outputs[-1][0].numpy())
        word += character_list[choice]
        input_sequence = one_hot_encode([choice], vocab_size)
    return word


max_epochs = 10  # passes through the data
for e in range(max_epochs):
    h = torch.zeros(batch_size, hiddenSize)
    for i in range(num_batches):
        x_in = X[i]
        y_in = y[i]

        out, h = gru(x, h)
        print(sample('Ma', 10))

{'h': 0, 'a': 1, 't': 2, 'M': 3}
[3, 1, 2, 0, 3, 1, 2, 0, 3, 1, 2, 0, 3, 1, 2, 0, 3, 1, 2, 0]
Weight Matrix for Wz:
 (tensor([[ 0.6614,  0.2669],
        [ 0.0617,  0.6213],
        [-0.4519, -0.1661],
        [-1.5228,  0.3817]], requires_grad=True), torch.Size([4, 2])) 

Weight Matrix for Uz:
 (tensor([[ 0.3255, -0.4791],
        [ 1.3790,  2.5286]], requires_grad=True), torch.Size([2, 2])) 

Weight Matrix for h_t_demo:
 (tensor([[0., 0.],
        [0., 0.]]), torch.Size([2, 2])) 

Bias vector for bz: 
 (tensor([0., 0.], requires_grad=True), torch.Size([2])) 

h0:tensor([[ 0.7565, -0.3472],
        [-0.1355, -0.2040]], grad_fn=<AddBackward0>)
h1:tensor([[-0.1535, -0.5712],
        [ 0.7664, -0.5062]], grad_fn=<AddBackward0>)
h2:tensor([[ 0.7495, -0.8616],
        [-0.2399, -0.6680]], grad_fn=<AddBackward0>)


RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.